In [1]:
#//////////////////////// Custom Functions for postprocess \\\\\\\\\\\\\\\\\\\\\\\\
def save_as_csv(out_path, contours):
    out_path = os.path.join(out_path, 'polygons.csv')
    print('save csv: %s, npoly = %d' % (out_path, len(contours)))
    fw = open(out_path, 'w')
    fw.write("filename,id,Geometry\n")
    for j, contour in enumerate(contours):
        polygon_str = re.sub(r"[\[\]]", '', ",".join(map(str, contour)))
        #polygon_str = polygon_str.replace(". ", ' ')
        #polygon_str = polygon_str.replace(".,", ',')
        #polygon_str = re.sub(r" {2,}", ' ', polygon_str)
        #polygon_str = re.sub(r" {0,}, {0,}", ',', polygon_str)

        # print(polygon_str)
        #points = polygon_str.split(",")
        #poly_str = ""
        #polygon_str = poly_str[:-1]
        fw.write("%s,%d,\"POLYGON ((%s))\"\n" % ("2022_09_12", j, polygon_str))
    fw.close()
    

'''
This function takes the contours made from pixels and map them to geo-coords.
The mapping is computed using cordinates of its corresponding raster.
'''
def assign_geocoords(contours,imgs_dir,img_path,y_flip=True):
    #img_path = get_respond_img(imgs_dir,npy_file)
    dataset = rasterio.open(img_path)

    x_min_coord = dataset.bounds[0] #left edge
    x_max_coord = dataset.bounds[2] #right edge
    y_min_coord = dataset.bounds[1] #bottom edge
    y_max_coord = dataset.bounds[3] #top edge
    
    y_img_size,x_img_size = dataset.read(1).shape
    dataset.close()

    for it in range(len(contours)):
        contour = contours[it]
        
        x = (contour[:,0])
        y = (contour[:,1])
        if(y_flip):
            y = y_img_size - (y)
            
        geo_x = x_min_coord + (x/x_img_size)*(x_max_coord - x_min_coord)
        geo_y = y_min_coord + (y/y_img_size)*(y_max_coord - y_min_coord)

        contours[it] = np.vstack((geo_x, geo_y)).T

    return contours



In [2]:
#//////////////////////// postprocess \\\\\\\\\\\\\\\\\\\\\\\\
import numpy as np
from scipy import ndimage as ndi
from skimage.segmentation import watershed
import matplotlib.pyplot as plt
from skimage import measure
import rasterio
from rasterio.features import shapes
from shapely.geometry import shape, Polygon
import re
import os


x_div = 2
y_div = 2

ct = 0
geoms_np = []
geoms_polygons = []
for x_tile in range(x_div):
    for y_tile in range(y_div):
        img = np.load("/home/gcf/Desktop/Ahmad/Building_Footprints_Extraction/Previous/arrays.npy")
        sizex, sizey = img.shape
        sizex_new, sizey_new = int(sizex/2), int(sizey/2)
        print(img.shape)
        img = img[x_tile*sizex_new:(x_tile+1)*sizex_new,y_tile*sizey_new:(y_tile+1)*sizey_new]
        print(img.shape)
        
        #plt.imsave("/home/gcf/Desktop/Ahmad/Building_Footprints_Extraction/Previous/1.png", npy_mean)
        M1 = (img >= 0.9999)
        startpts, _ = ndi.label(M1)
        print("pts done")
        #plt.imsave("/home/gcf/Desktop/Ahmad/Building_Footprints_Extraction/Previous/2.png", startpts)
        ws_out = watershed(image=(-1*img), markers=startpts, mask=(img>0), watershed_line=True)
        print("watershed done")

        #plt.imsave("/home/gcf/Desktop/Ahmad/Building_Footprints_Extraction/Previous/3.png", ws_out)
        labels = measure.label(ws_out, connectivity=2, background=0).astype('uint16')
        polygon_gen = shapes(labels, labels>0)
        for polygon, value in polygon_gen:
            ct = ct+1
            '''
            try:
                p = np.array(polygon['coordinates']).reshape(-1,2).astype(int)
            except:
                p = np.array(polygon['coordinates']).reshape(-1,2)
                print(p)
            '''
            #print(p)
            #print(p)
            p = shape(polygon)
            if p.area >= 25:
                p = p.simplify(tolerance=0.5)
                geoms_polygons.append(p)
                try:
                    p = np.array(p.boundary.xy, dtype='int32').T
                except:
                    p = np.array(p.boundary[0].xy, dtype='int32').T
                
                pcoords_a = y_tile*sizey_new + p[:,0]
                pcoords_b = x_tile*sizex_new + p[:,1]
                p = np.vstack((pcoords_a,pcoords_b)).T
                geoms_np.append(p)
            #p = re.sub('[\[\]]',''," ".join(map(str, p)))
            #print(p)
            #fw.write("%d,%s\n" % (ct, p))
        
        # Clear out RAM
        img = None
        M1 = None
        startpts = None
        ws_out = None
        labels = None
        polygon_gen = None
        
geo_poly = (assign_geocoords(geoms_np, "", r"/home/gcf/Desktop/Ahmad/Building_Footprints_Extraction/zoom21_DHA.tif"))
save_as_csv(r"/home/gcf/Desktop/Ahmad/Building_Footprints_Extraction", geo_poly)
#np.save("/home/gcf/Desktop/Ahmad/Building_Footprints_Extraction/Previous/labels.npy", labels)
print("Complete")

(81920, 114688)
(40960, 57344)
pts done
watershed done


/home/gcf/.local/lib/python3.6/site-packages/ipykernel_launcher.py:57: ShapelyDeprecationWarning: __getitem__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.


(81920, 114688)
(40960, 57344)
pts done
watershed done
(81920, 114688)
(40960, 57344)
pts done
watershed done
(81920, 114688)
(40960, 57344)
pts done
watershed done
save csv: /home/gcf/Desktop/Ahmad/Building_Footprints_Extraction/polygons.csv, npoly = 108995
Complete
